In [ ]:
import numpy as np
import visa
import time
GateStep = 1####
GateDelay = 0.05
# initialize k2400.
def initE3640A(instr):
    rm = visa.ResourceManager()
    E3640A = rm.open_resource(instr)
    # set various k2400 parameters; see k2400 manual for the full selection
    E3640A.write('SOUR:VOLT 0') # set voltage to zero
    E3640A.write('SOUR:VOLT:RANG 20') # set voltage range
    #k2400.write('FORM:ELEM VOLT, CURR') # set communications formatting 
    E3640A.write('OUTP ON') # turn on output
    E3640A.write('CURR:PROT 100e-3') # set current compliance
    return E3640A

# read dc current from k2400
def IreadE3640A(E3640A):
    return E3640A.query_ascii_values("MEAS:CURR?")[0]
def VreadE3640A(E3640A):
    return E3640A.query_ascii_values("MEAS:VOLT?")[0]


# set dc voltage
def setE3640A(E3640A,target_v):
    return E3640A.write('SOUR:VOLT %1.6f'%target_v)

# sweep the dc voltage to target_v in small voltage steps with a specified delay.
# differs from set2400, which sets voltage as fast as the instrument will go.
def sweepE3640A(E3640A,target_v):
    step  = GateStep
    delay = GateDelay
    cur_v = VreadE3640A(E3640A)
    while abs(cur_v-target_v) > step:
        if cur_v < target_v:
            setE3640A(E3640A,cur_v+step)
        else:
            setE3640A(E3640A,cur_v-step)
        time.sleep(delay)
        cur_v = VreadE3640A(E3640A)
    setE3640A(E3640A,target_v)